In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

VERSION = "v2.2"
OUTPUT_FORMAT = "json"
API_KEY = "e3e7698185a74e2f87a38a87d6b4f705"

In [3]:
def cumtdRequestUrl(methodname, other_args={}, version=VERSION, output=OUTPUT_FORMAT, key=API_KEY):
    rooturl = "https://developer.cumtd.com/api/{v}/{f}/".format(v=VERSION, f=OUTPUT_FORMAT)
    url = rooturl + methodname + "?key={}".format(key)
    for param, value in other_args.items():
        url = url + "&{}={}".format(param, value)
    return url

In [4]:
requests.get(cumtdRequestUrl("getapiusage")).json()

{'time': '2018-10-30T14:54:45-05:00',
 'new_changeset': True,
 'status': {'code': 200, 'msg': 'ok'},
 'rqst': {'method': 'GetApiUsage', 'params': {}},
 'days': []}

In [5]:
# get all stops
r = requests.get(cumtdRequestUrl("getstops"))
json = r.json()

In [6]:
# arrange into csv
def generateAllStopsCsv(filename):
    r = requests.get(cumtdRequestUrl("getstops"))
    json = r.json()
    
    stops = {'stop_id': [], 'stop_name': [], 'code': [], 'distance': [], 'specific_stop_code': [], 'specific_stop_stop_id': [], 'specific_stop_stop_lat': [], 'specific_stop_stop_lon': [], 'specific_stop_stop_name': []}

    for stop in json['stops']:
        for key in stop.keys():
            if key in ['stop_id', 'stop_name', 'code', 'distance']:
                stops[key].append(stop[key])
            elif key == 'stop_points': pass
        for key in ['specific_stop_code', 'specific_stop_stop_id', 'specific_stop_stop_lat', 'specific_stop_stop_lon', 'specific_stop_stop_name']:
            stops[key].append('')
        for specific_stop in stop['stop_points']:
            for key in specific_stop.keys():
                if key in ['code', 'stop_id', 'stop_lat', 'stop_lon', 'stop_name']:
                    stops['specific_stop_' + key].append(specific_stop[key])
            for key in ['stop_id', 'stop_name', 'code', 'distance']:
                stops[key].append('')

    pd.DataFrame(stops).to_csv(filename, index=False)
    
# generateAllStopsCsv('all_stops.csv')

In [39]:
def nameToStopId(name):
    stops = pd.read_csv('all_stops.csv')
    if name in list(stops['stop_name']):
        return stops[stops['stop_name'] == name].iloc[0]['stop_id']
    elif name in list(stops['specific_stop_stop_name']):
        return stops[stops['specific_stop_stop_name'] == name].iloc[0]['specific_stop_stop_id']
    else:
        return None
    
nameToStopId('U.S. 150 and Dale')

'150DALE'

In [79]:
STOP = nameToStopId('Wright & Springfield (SW Corner)')
r = requests.get(cumtdRequestUrl("getdeparturesbystop", {'stop_id': STOP, 'pt': 60}))
json = r.json()

print(json['status']['msg'])

ok


In [82]:
names = []
scheduled = []
expected = []
diff = []

for departure in json['departures']:
    names.append(departure['headsign'])
    scheduled_time = datetime.fromisoformat(departure['scheduled'])
    expected_time = datetime.fromisoformat(departure['expected'])
    diff.append(expected_time - scheduled_time)
    scheduled.append(scheduled_time)
    expected.append(expected_time)
    
df = pd.DataFrame({'name': names, 'scheduled_time': scheduled, 'expected_time': expected, 'diff': diff})
df

,name,scheduled_time,expected_time,diff


In [70]:
stop_times_all = pd.read_csv('google_transit/stop_times.txt')
stop_times_all.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type
0,R1SATPO__R1SA,07:08:00,07:08:00,DEPOT:1,0,Champaign Meijer,0,0
1,R1SATPO__R1SA,07:13:00,07:13:00,IU:2,2,Champaign Meijer,0,0
2,20RED001__R1SA,07:13:00,07:13:00,IU:2,0,Champaign Meijer,0,0
3,20RED001__R1SA,07:14:00,07:14:00,WRTHLY:2,1,NaN,0,0
4,20RED001__R1SA,07:15:30,07:15:30,WHTWRT:2,2,NaN,0,0


In [78]:
trip_id = '[@2.0.80548152@][12][1425572286750]/26__BB2_MF'
departure_time = '15:29:00'
expected_time = '15:29:06'
day = '10/30'
stop_times_all.loc[(stop_times_all['trip_id'] == trip_id) & (stop_times_all['departure_time'] == departure_time),day] = expected_time
stop_times_all[(stop_times_all['trip_id'] == trip_id) & (stop_times_all['departure_time'] == departure_time)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,day,10/30
93053,[@2.0.80548152@][12][1425572286750]/26__BB2_MF,15:29:00,15:29:00,PLAZA:2,70,NaN,0,0,10/30,15:29:06
